In [32]:
import pandas as pd
import fba_utils
from fba_utils import get_flux_bounds, convert_transcriptomics_to_enzyme_activity
import cobra
import numpy as np
import os 

In [3]:
os.getcwd()

'/home/connah/Documents/Repos/syn_bmca'

## 1) collect input

In [27]:
# get the cobra model for S. elongatus
syn_model = cobra.io.read_sbml_model('models/syn_elong.xml')
[m.id for m in syn_model.metabolites if m.id.startswith('sucr') ]

['sucr_c', 'sucr_e']

In [11]:
# get the metabolomics data, this is only for sucrose activity/abundance
metab_df = pd.read_csv('data/circadian_experiments/processed_data/sucrose_metabolomics.csv',index_col=0)
display(metab_df)

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
sucr,0.644,0.654,0.681,0.881,0.811,1.031,0.829,1.006,0.964,1.811,...,0.788,2.17,2.231,2.27,2.404,3.216,3.098,4.431,3.103,4.188


In [10]:
# get the transcriptomics data 
transcript_df = pd.read_csv('data/circadian_experiments/raw_data/Se_Sucrose_circadian_data/Counts_ESeq2median_norm.csv',index_col=0)
display(transcript_df)

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
SYNPCC7942_RS00005,607.927550,582.795478,551.827288,975.630070,884.915418,822.426446,975.113687,961.357516,1005.261196,1031.011880,...,903.383312,1319.278749,1446.428293,962.039815,1201.548782,1200.140721,1160.674229,951.044468,944.250567,923.493988
SYNPCC7942_RS00010,302.461483,289.783347,286.620741,262.244540,260.796220,223.922884,237.446515,220.505547,248.107019,157.592774,...,439.786605,399.962336,431.345580,335.576388,316.650821,267.897315,263.509184,184.590910,201.161646,184.045382
SYNPCC7942_RS00015,2057.138694,1768.566334,1912.452184,2982.152470,2965.312745,2368.216681,2620.354206,2786.070083,2380.544066,2374.989692,...,1315.858328,2560.100067,2721.523710,2326.283777,2744.045848,2624.019860,2781.387903,3281.170296,2854.736581,3055.806756
SYNPCC7942_RS00020,4618.046018,4923.088121,3877.616229,3263.487609,3846.246542,3090.548566,2103.248462,2291.390973,2263.976545,1843.391532,...,1503.537962,1682.570764,1864.859335,2109.337298,1793.309598,2115.701876,2062.245784,3516.590587,3250.464409,2721.475796
SYNPCC7942_RS00025,276.421753,274.446624,289.915232,429.036087,653.981361,803.852198,639.522613,803.853554,928.262466,910.042779,...,654.077530,600.369903,677.582778,723.967462,573.733665,708.504988,825.779615,272.873518,286.902675,326.707779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HTX97_RS00025,17.025978,17.758311,17.296079,25.119209,60.719731,63.977967,28.493582,44.334449,28.874524,28.855015,...,31.513371,54.579082,35.299738,33.313879,29.000199,37.289736,40.539874,36.115613,24.183367,21.780519
HTX97_RS00030,45.068764,43.588581,36.239404,62.295638,73.660001,65.009870,61.208435,73.501849,66.304462,39.953098,...,46.219611,67.371054,68.016569,56.877354,35.270512,53.971987,60.809812,41.466074,36.275051,35.937856
HTX97_RS00035,3.004584,0.000000,0.000000,4.019073,1.990811,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.400594,0.852798,2.582908,0.812534,0.783789,1.962618,0.881302,1.337615,1.099244,3.267078
HTX97_RS00040,7.010697,12.107939,16.472456,17.081062,4.977027,9.287124,5.276589,16.333744,7.485988,9.988274,...,18.207726,20.467156,13.775508,6.500269,7.837892,17.663559,8.813016,10.700922,5.496220,5.445130


## 2) Calculate additional data

In [ ]:
# calculate rates



In [28]:
# calculate enzyme activity
# Run enzyme activity converter for all strains in transcriptomics_df
all_enzyme_activity_df = convert_transcriptomics_to_enzyme_activity(transcript_df, syn_model)
display(all_enzyme_activity_df)


,Reaction_ID,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
EX_gln__L_e: gln__L_e -->,EX_gln__L_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EX_hco3_e: hco3_e <=>,EX_hco3_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EX_mn2_e: mn2_e <=>,EX_mn2_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EX_arg__L_e: arg__L_e -->,EX_arg__L_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADPT: ade_c + prpp_c <=> amp_c + ppi_c,ADPT,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZDS: h_c + nadph_c + o2_c + zcarote_c --> 2.0 h2o_c + nadp_c + norsp_c,ZDS,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
RDXRr: nadh_c + 2.0 rdxo_c <=> h_c + nad_c + 2.0 rdxr_c,RDXRr,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
MPTSS: atp_c + h_c + moadcoo_c --> moadamp_c + ppi_c,MPTSS,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
NOR: h_c + nadph_c + norsp_c + o2_c --> 2.0 h2o_c + lyc_c + nadp_c,NOR,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


In [ ]:
# check the enzyme conversion
all_enzyme_activity_df['L_T0_A'].dropna().min()

In [33]:
### Cleanup enzyme activity df
clean_enzyme_activity_df = all_enzyme_activity_df.copy()

# Replace current index with Reaction_ID column
clean_enzyme_activity_df = clean_enzyme_activity_df.set_index('Reaction_ID')

# Drop rows with all 0s: 
clean_enzyme_activity_df = clean_enzyme_activity_df.loc[~(clean_enzyme_activity_df==0).all(axis=1)]

# Drop rows with all infs:
clean_enzyme_activity_df.replace([np.inf, -np.inf], np.nan, inplace=True)
clean_enzyme_activity_df.dropna(how='all', inplace=True)

clean_enzyme_activity_df

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
Reaction_ID,,,,,,,,,,,,,,,,,,,,,
DTMPK,178.272000,156.596015,187.786003,160.762936,123.430272,114.541199,132.970048,140.003522,116.567522,169.800665,...,207.988251,185.909998,149.808645,138.130717,172.433615,116.775753,138.364354,145.800066,195.665426,149.196552
ACOATA,404.617348,452.836930,367.335777,583.770413,636.064063,599.535465,566.705682,541.346951,544.338243,432.825224,...,293.424502,569.669169,588.902948,576.898876,582.355346,525.000234,539.356590,583.200265,533.133324,503.129979
3HAD160,241.368269,171.125542,172.137169,183.872609,116.462434,99.062659,165.684901,131.836650,104.803827,178.679131,...,280.118856,264.367429,224.712967,82.878430,169.298459,158.972033,155.109085,268.860673,193.466938,235.229601
RNDR3,2704.125840,2629.037221,2801.141205,5320.248426,6440.273066,6515.433605,4435.500897,4978.291895,4880.863935,5450.268400,...,1703.122647,5425.501875,4777.518212,6622.961601,6265.610544,7346.078037,7799.519310,10922.966420,5126.873860,4654.496820
G5SD,1862.842245,1853.321909,1853.974964,1931.164774,1893.261108,1781.064048,1973.444367,2080.218994,1932.454235,1725.751855,...,2043.467058,2109.822641,2228.188346,1980.956985,2197.744805,2174.580408,2157.426358,1872.661400,1917.081478,1769.667134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MDDCP5pp,862.315684,912.938624,885.394530,917.353506,1001.377852,1238.283232,1020.492355,1135.195222,1150.703241,1019.913798,...,1110.671266,905.671643,903.156714,1032.730241,814.356937,833.131212,842.524346,741.038868,718.905554,824.392628
MDDEP2pp,2088.186065,2206.066541,2120.005135,1920.112322,2121.208949,2432.194648,2667.843509,2781.403299,2508.875283,1981.007757,...,2303.277298,2323.022180,2327.199807,2145.901311,1570.713476,1817.383987,1816.362633,1322.901518,1172.893312,1275.249363
MDDEP3pp,2088.186065,2206.066541,2120.005135,1920.112322,2121.208949,2432.194648,2667.843509,2781.403299,2508.875283,1981.007757,...,2303.277298,2323.022180,2327.199807,2145.901311,1570.713476,1817.383987,1816.362633,1322.901518,1172.893312,1275.249363


11.01680898

## 3) Normalize dataframes with respect to reference condition

In [21]:
# define reference condition
ref_cond = [c for c in metab_df.columns if 'T16' in c and 'B' in c][0]
display(ref_cond)

'L_T16_B'

In [22]:
# normalize metabolomics data
norm_metab_df = metab_df.div(metab_df[ref_cond],axis=0)
display(norm_metab_df)
display(norm_metab_df[ref_cond])

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
sucr,0.060296,0.061232,0.06376,0.082486,0.075932,0.09653,0.077617,0.094189,0.090257,0.169559,...,0.073778,0.203172,0.208883,0.212534,0.225081,0.301106,0.290058,0.414864,0.290526,0.392112


sucr    1.0
Name: L_T16_B, dtype: float64

In [34]:
# normalize transcriptomics data
norm_enzyme_activity_df = clean_enzyme_activity_df.div(clean_enzyme_activity_df[ref_cond],axis=0)
display(norm_enzyme_activity_df)
display(norm_enzyme_activity_df[ref_cond])



,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
Reaction_ID,,,,,,,,,,,,,,,,,,,,,
DTMPK,1.118515,0.982515,1.178208,1.008659,0.774427,0.718655,0.834281,0.878411,0.731368,1.065364,...,1.304961,1.166437,0.939930,0.866660,1.081884,0.732675,0.868126,0.914779,1.227645,0.936090
ACOATA,0.991527,1.109691,0.900168,1.430547,1.558694,1.469180,1.388730,1.326587,1.333918,1.060651,...,0.719046,1.395992,1.443125,1.413708,1.427080,1.286529,1.321710,1.429150,1.306460,1.232935
3HAD160,1.163375,0.824811,0.829687,0.886251,0.561339,0.477474,0.798588,0.635442,0.505146,0.861219,...,1.350150,1.274229,1.083098,0.399467,0.816005,0.766232,0.747613,1.295886,0.932495,1.133787
RNDR3,0.292695,0.284568,0.303196,0.575865,0.697097,0.705232,0.480100,0.538852,0.528306,0.589939,...,0.184346,0.587258,0.517120,0.716871,0.678191,0.795142,0.844222,1.182305,0.554934,0.503804
G5SD,1.114043,1.108350,1.108740,1.154903,1.132235,1.065137,1.180187,1.244042,1.155674,1.032059,...,1.222063,1.261746,1.332533,1.184680,1.314327,1.300473,1.290215,1.119916,1.146480,1.058321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MDDCP5pp,0.814008,0.861795,0.835794,0.865962,0.945279,1.168913,0.963323,1.071600,1.086239,0.962777,...,1.048450,0.854935,0.852561,0.974875,0.768736,0.786458,0.795325,0.699525,0.678632,0.778209
MDDEP2pp,1.393032,1.471670,1.414258,1.280910,1.415061,1.622520,1.779722,1.855478,1.673674,1.321533,...,1.536519,1.549691,1.552478,1.431534,1.047825,1.212379,1.211698,0.882509,0.782439,0.850721
MDDEP3pp,1.393032,1.471670,1.414258,1.280910,1.415061,1.622520,1.779722,1.855478,1.673674,1.321533,...,1.536519,1.549691,1.552478,1.431534,1.047825,1.212379,1.211698,0.882509,0.782439,0.850721


Reaction_ID
DTMPK       1.0
ACOATA      1.0
3HAD160     1.0
RNDR3       1.0
G5SD        1.0
           ... 
MDDCP5pp    1.0
MDDEP2pp    1.0
MDDEP3pp    1.0
MDDEP4pp    1.0
MNHNAtpp    1.0
Name: L_T16_B, Length: 682, dtype: float64

## 4) Save to processed data

In [36]:
norm_metab_df_fname = 'data/circadian_experiments/processed_data/normalized_sucrose_metabolomics.csv'
norm_enzyme_activity_df_fname = 'data/circadian_experiments/processed_data/normalized_enzyme_activity.csv'
enzyme_activity_df_fname = 'data/circadian_experiments/processed_data/cleaned_enzyme_activity.csv'

norm_metab_df.to_csv(norm_metab_df_fname)

norm_enzyme_activity_df.to_csv(norm_enzyme_activity_df_fname)

clean_enzyme_activity_df.to_csv(enzyme_activity_df_fname)
